# Ray et al 2013 Training 
**Authorship:**
Adam Klie, *08/31/2022*
***
**Description:**
Notebook to perform simple training of *single task* and *multitask* models on the Ray et al (2013) dataset.
Also take a look at the `ray13_training_ST.py` script for usage. The script was run because all 244 models took several hours to train.
***

In [11]:
if 'autoreload' not in get_ipython().extension_manager.loaded:
    %load_ext autoreload
%autoreload 2

import os
import logging
import torch
import numpy as np
import pandas as pd
import eugene as eu

In [12]:
eu.settings.dataset_dir = "/cellar/users/aklie/data/eugene/ray13"
eu.settings.output_dir = "/cellar/users/aklie/projects/EUGENe/EUGENe_paper/output/ray13"
eu.settings.logging_dir = "/cellar/users/aklie/projects/EUGENe/EUGENe_paper/logs/ray13"
eu.settings.config_dir = "/cellar/users/aklie/projects/EUGENe/EUGENe_paper/configs/ray13"
eu.settings.verbosity = logging.ERROR

# Load in the SetA training `SeqData`'s for single task and multi-task models

In [13]:
# Load in the training SetA processed data for single task and multitask models
sdata_training_ST = eu.dl.read_h5sd(os.path.join(eu.settings.dataset_dir, eu.settings.dataset_dir, "norm_setA_processed_ST.h5sd"))
sdata_training_MT = eu.dl.read_h5sd(os.path.join(eu.settings.dataset_dir, eu.settings.dataset_dir, "norm_setA_processed_MT.h5sd"))

In [14]:
sdata_training_ST, sdata_training_MT

(SeqData object with = 120326 seqs
 seqs = (120326,)
 names = (120326,)
 rev_seqs = None
 ohe_seqs = (120326, 4, 41)
 ohe_rev_seqs = None
 seqs_annot: 'Probe_Set', 'RNCMPT00001', 'RNCMPT00002', 'RNCMPT00003', 'RNCMPT00004', 'RNCMPT00005', 'RNCMPT00006', 'RNCMPT00007', 'RNCMPT00008', 'RNCMPT00009', 'RNCMPT00010', 'RNCMPT00011', 'RNCMPT00012', 'RNCMPT00013', 'RNCMPT00014', 'RNCMPT00015', 'RNCMPT00016', 'RNCMPT00017', 'RNCMPT00018', 'RNCMPT00019', 'RNCMPT00020', 'RNCMPT00021', 'RNCMPT00022', 'RNCMPT00023', 'RNCMPT00024', 'RNCMPT00025', 'RNCMPT00026', 'RNCMPT00027', 'RNCMPT00028', 'RNCMPT00029', 'RNCMPT00031', 'RNCMPT00032', 'RNCMPT00033', 'RNCMPT00034', 'RNCMPT00035', 'RNCMPT00036', 'RNCMPT00037', 'RNCMPT00038', 'RNCMPT00039', 'RNCMPT00040', 'RNCMPT00041', 'RNCMPT00042', 'RNCMPT00043', 'RNCMPT00044', 'RNCMPT00045', 'RNCMPT00046', 'RNCMPT00047', 'RNCMPT00049', 'RNCMPT00050', 'RNCMPT00051', 'RNCMPT00052', 'RNCMPT00053', 'RNCMPT00054', 'RNCMPT00055', 'RNCMPT00056', 'RNCMPT00057', 'RNCMPT0005

In [5]:
# Grab the prediction columns for single task and multitask
target_mask_ST = sdata_training_ST.seqs_annot.columns.str.contains("RNCMPT")
target_cols_ST = sdata_training_ST.seqs_annot.columns[target_mask_ST]
target_mask_MT = sdata_training_MT.seqs_annot.columns.str.contains("RNCMPT")
target_cols_MT = sdata_training_MT.seqs_annot.columns[target_mask_MT]

In [6]:
target_cols_MT[215]

'RNCMPT00268'

# Train single task models

In [3]:
# Instantiation function
from pytorch_lightning import seed_everything
def prep_new_model(
    seed,
    conv_dropout = 0,
    fc_dropout = 0,
    batchnorm = True
):
    model = eu.models.DeepBind(
        input_len=41, # Length of padded sequences
        output_dim=1, # Number of multitask outputs
        strand="ss",
        task="regression",
        conv_kwargs=dict(channels=[4, 16], conv_kernels=[16], dropout_rates=conv_dropout, batchnorm=batchnorm),
        mp_kwargs=dict(kernel_size=8),
        fc_kwargs=dict(hidden_dims=[32], dropout_rate=fc_dropout, batchnorm=batchnorm),
        optimizer="sgd",
        lr=0.0005,
        scheduler_patience=3
    )

    # Set a seed
    seed_everything(seed)
    
    # Initialize the model prior to conv filter initialization
    eu.models.init_weights(model)

    # Return the model
    return model 

In [7]:
# Test out a model before training
model = prep_new_model(0)
print(model.summary())
sdataloader = sdata_training_ST[:64].to_dataset().to_dataloader()
test_seqs = next(iter(sdataloader))
print(model(test_seqs[1], test_seqs[2]).size())

Global seed set to 0


Model: DeepBind
Input length: 41
Output dimension: 1
Strand: ss
Task: regression
Aggregation: max
Loss function: mse_loss
Optimizer: sgd
	Optimizer parameters: {}
Learning rate: 0.0005
Scheduler: lr_scheduler
Scheduler patience: 3
  | Name      | Type                      | Params
--------------------------------------------------------
0 | hp_metric | R2Score                   | 0     
1 | convnet   | BasicConv1D               | 1.1 K 
2 | max_pool  | MaxPool1d                 | 0     
3 | avg_pool  | AvgPool1d                 | 0     
4 | fcn       | BasicFullyConnectedModule | 1.2 K 
--------------------------------------------------------
2.2 K     Trainable params
0         Non-trainable params
2.2 K     Total params
0.009     Total estimated model params size (MB)
No transforms given, assuming just need to tensorize.
torch.Size([64, 1])


In [18]:
# Train a model on each target prediction!
for i, target_col in enumerate(target_cols_ST):
    print(f"Training DeepBind SingleTask model on {target_col}")

    # Initialize the model
    model = prep_new_model(seed=i, conv_dropout=0.5, fc_dropout=0.5, batchnorm=True)

    # Train the model
    eu.train.fit(
        model=model, 
        sdata=sdata_training_ST, 
        gpus=1, 
        target_keys=target_col,
        train_key="train_val",
        epochs=5,
        early_stopping_metric="val_loss",
        early_stopping_patience=3,
        batch_size=64,
        num_workers=0,
        name="DeepBind_ST",
        seed=i,
        version=target_col,
        verbosity=logging.ERROR
    )
    
    # Get predictions on the training data
    eu.evaluate.train_val_predictions(
        model,
        sdata=sdata_training_ST, 
        target_keys=target_col,
        train_key="train_val",
        batch_size=1024,
        num_workers=0,
        name="DeepBind_ST",
        suffix="_ST",
        version=target_col
    )
    del model 
#sdata_training_ST.write_h5sd(os.path.join(eu.settings.output_dir, "DeepBind_ST", "norm_training_predictions_ST.h5sd"))

Global seed set to 0
Global seed set to 0


Training DeepBind SingleTask model on RNCMPT00001
Dropping 610 sequences with NaN targets.
No transforms given, assuming just need to tensorize.


GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Set SLURM handle signals.

  | Name      | Type                      | Params
--------------------------------------------------------
0 | hp_metric | R2Score                   | 0     
1 | convnet   | BasicConv1D               | 1.1 K 
2 | max_pool  | MaxPool1d                 | 0     
3 | avg_pool  | AvgPool1d                 | 0     
4 | fcn       | BasicFullyConnectedModule | 1.2 K 
--------------------------------------------------------
2.2 K     Trainable params
0         Non-trainable params
2.2 K     Total params
0.009     Total estimated model params size (MB)


No transforms given, assuming just need to tensorize.


Validation sanity check: 0it [00:00, ?it/s]

...

No transforms given, assuming just need to tensorize.


Predicting: 0it [00:00, ?it/s]

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting: 0it [00:00, ?it/s]

SeqData object modified:
    seqs_annot:
        + RNCMPT00002_predictions_ST


# Train multi-task model

In [5]:
# Define the version for saving
model_version = 0

In [9]:
# Instantiate the model
conv_dropout = 0.25
fc_dropout = 0.25
batchnorm = True
model = eu.models.DeepBind(
    input_len=41, # Length of padded sequences
    output_dim=len(target_cols_MT), # Number of multitask outputs
    strand="ss", # Strand information to include, only forward strand
    task="regression", # Task type, regression in this case
    optimizer="adam", # Optimizer to use
    optimizer_kwargs={}, # Default optimizer kwargs
    lr=0.0005, # Learning rate to start with
    scheduler_patience=2, # Number of epochs to wait before reducing learning rate
    conv_kwargs=dict(channels=[4, 1024], conv_kernels=[16], dropout_rates=conv_dropout, batchnorm=batchnorm), # Convolutional layer kwargs
    fc_kwargs=dict(hidden_dims=[512], dropout_rate=fc_dropout, batchnorm=batchnorm) # Fully connected layer kwargs
)
model.summary(), model_version

Model: DeepBind
Input length: 41
Output dimension: 233
Strand: ss
Task: regression
Aggregation: max
Loss function: mse_loss
Optimizer: adam
	Optimizer parameters: {}
Learning rate: 0.0005
Scheduler: lr_scheduler
Scheduler patience: 2


NameError: name 'model_version' is not defined

In [7]:
# Train the model
eu.train.fit(
    model=model,
    sdata=sdata_training_MT,
    gpus=1,
    target_keys=target_cols_MT,
    train_key="train_val",
    epochs=100,
    early_stopping_metric="val_loss",
    early_stopping_patience=5,
    batch_size=1024,
    num_workers=0,
    name="DeepBind_MT",
    seed=42,
    version=f"v{model_version}",
    verbosity=logging.ERROR
)

# Get predictions on the training data
eu.evaluate.train_val_predictions(
    model,
    sdata=sdata_training_MT, 
    target_keys=target_cols_MT,
    train_key="train_val",
    batch_size=1024,
    num_workers=0,
    name="DeepBind_MT",
    suffix="_MT",
    version=f"v{model_version}"
)

Global seed set to 42


Dropping 0 sequences with NaN targets.
No transforms given, assuming just need to tensorize.


GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


No transforms given, assuming just need to tensorize.


Set SLURM handle signals.

  | Name      | Type                      | Params
--------------------------------------------------------
0 | hp_metric | R2Score                   | 0     
1 | convnet   | BasicConv1D               | 68.6 K
2 | max_pool  | MaxPool1d                 | 0     
3 | avg_pool  | AvgPool1d                 | 0     
4 | fcn       | BasicFullyConnectedModule | 1.2 M 
--------------------------------------------------------
1.2 M     Trainable params
0         Non-trainable params
1.2 M     Total params
4.953     Total estimated model params size (MB)


...

Predicting: 0it [00:00, ?it/s]

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting: 0it [00:00, ?it/s]

/cellar/users/aklie/opt/miniconda3/envs/eugene_dev/lib/python3.7/site-packages/pandas/core/frame.py:3636: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider using pd.concat instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  self[k1] = value[k2]


SeqData object modified:
    seqs_annot:
        + RNCMPT00017_predictions_MT, RNCMPT00075_predictions_MT, RNCMPT00142_predictions_MT, RNCMPT00146_predictions_MT, RNCMPT00255_predictions_MT, RNCMPT00074_predictions_MT, RNCMPT00291_predictions_MT, RNCMPT00073_predictions_MT, RNCMPT00093_predictions_MT, RNCMPT00287_predictions_MT, RNCMPT00147_predictions_MT, RNCMPT00088_predictions_MT, RNCMPT00116_predictions_MT, RNCMPT00113_predictions_MT, RNCMPT00079_predictions_MT, RNCMPT00064_predictions_MT, RNCMPT00217_predictions_MT, RNCMPT00150_predictions_MT, RNCMPT00024_predictions_MT, RNCMPT00141_predictions_MT, RNCMPT00235_predictions_MT, RNCMPT00022_predictions_MT, RNCMPT00256_predictions_MT, RNCMPT00153_predictions_MT, RNCMPT00253_predictions_MT, RNCMPT00215_predictions_MT, RNCMPT00240_predictions_MT, RNCMPT00176_predictions_MT, RNCMPT00059_predictions_MT, RNCMPT00027_predictions_MT, RNCMPT00225_predictions_MT, RNCMPT00023_predictions_MT, RNCMPT00122_predictions_MT, RNCMPT00178_predictions_M

In [8]:
# Save the predictions!
sdata_training_MT.write_h5sd(os.path.join(eu.settings.output_dir, "DeepBind_MT", f"norm_training_predictions_v{model_version}_MT.h5sd"))

In [9]:
# Double check we predicted on all the columns
np.sum(sdata_training_MT.seqs_annot.columns.str.contains("RNCMPT"))

466

In [16]:
# Move on to the next model version if training multiple
model_version = model_version + 1

---